# Identifying Optimal Locations for Homeless Shelters in San Diego

### By: Hede Yang: A16635784 Teresa Lee: A16327096


### Question(s) you addressed, why it is important:

We are looking for the best location of building homeless shelters in San Diego. Strategically placing homeless shelters in areas with lower crime rates and within promise zones, we can create safer environments for both the homeless population and the surrounding communities. We expect to find correlations between crime rates, drug use, promise zones, and the presence of homeless populations, indicating potential areas where the establishment of shelters could have the most significant positive impact.

We hypothesize that there are significant correlations between crime rates, poverty, and the presence of homeless populations. Identifying these correlations can reveal potential areas where establishing shelters and more resources could have the most significant positive impact.

We aim to provide actionable insights for local governments, non-profit organizations, and other stakeholders. By optimizing shelter placement, we can enhance public safety, reduce homelessness, and improve community well-being.

Throughout this project, we will aim to identify best locations to built homeless shelters and see if our analysis matches with what the government is currently working towards. 

## Background and literature:

The article from CBS8 reports findings by the San Diego County District Attorney's Office showing that homeless individuals in San Diego are significantly more likely to be involved in crimes, both as victims and perpetrators, compared to the rest of the population. This context directly informs our research question on identifying the best areas to build homeless shelters, emphasizing the need for shelters as part of a broader strategy to reduce crime and improve safety in homeless populations.
https://www.cbs8.com/article/news/crime/da-homeless-more-likely-crime-victims-and-perpetrators-sd-county/509-c01ca175-004b-483e-93b1-1b2c66afd5f1


This is a page with pre-made maps on the homelessness issues in San Diego. The maps are created for people to do analysis. We can use this page to help compare with the maps we get from ArcGIS (https://storymaps.arcgis.com/stories/53ae35abaf7746298ec713e750921078)

This article talks about the current homeless population issues in San Diego. It is also interesting to note that the homeless population in downtown actually hit a two-year low recently as there has been efforts to enforce the city’s camping ban and push for new shelters. We hope that with more analysis on this will help us figure out best places to set up new shelters to even further minimize the homeless population. (https://www.sandiegouniontribune.com/news/homelessness/story/2023-12-11/the-homeless-population-downtown-just-hit-a-two-year-low-thats-good-and-bad-news)



## Python libraries or ArcGIS modules you plan to use and why:
We are planning on using the ArcGIS API for Python. The ArcGIS API for Python provides powerful spatial analysis capabilities and integration with ArcGIS Online. 
The ArcGIS API for Python allows us to easily access various geospatial datasets hosted on ArcGIS Online, including crime rate data, promise zone boundaries, and other relevant layers such as population density, transportation networks, and land use.
Spatial Analysis: With the ArcGIS API for Python, we can perform spatial analysis to identify suitable locations for homeless shelters. This may include proximity analysis to identify areas with low crime rates and within promise zones, as well as other spatial operations such as overlay analysis to identify areas with the highest concentration of homeless individuals.
Mapping and Visualization: The API provides capabilities for creating interactive maps and visualizations, allowing us to communicate our findings effectively. We can create thematic maps to visualize crime rates, promise zones, and potential shelter locations, providing valuable insights for decision-makers.
Integration with ArcGIS Online: The ArcGIS API for Python seamlessly integrates with ArcGIS Online, allowing us to publish and share our analysis results as web maps and applications. This facilitates collaboration and decision-making among stakeholders involved in addressing homelessness in San Diego.


In [2]:
import warnings
warnings.filterwarnings("ignore")

import arcgis
from arcgis.gis import GIS
from arcgis.raster.functions import *
from ipywidgets import *
from IPython.display import display

# Importing graphviz would be useful as we trace layer operations
import graphviz

import pandas as pd
from arcgis.geocoding import Geocoder, get_geocoders, geocode
import geopandas
from arcgis.geometry import Point, Polyline, Polygon


from arcgis.geoenrichment import *
from arcgis.features.enrich_data import enrich_layer
from arcgis.features.analyze_patterns import find_hot_spots
from arcgis.features.manage_data import overlay_layers

import seaborn as sns
import matplotlib.pyplot as plt



## Connect to GIS

In [3]:
import getpass
username = input('Enter username: ')
password = getpass.getpass("Enter password: ")
agol = GIS(username=username, password=password)

Enter username: dsc170wi24_52
Enter password: ········


## Data Sources: 
Point data of all encampment-categorized: 
https://ucsdonline.maps.arcgis.com/home/search.html?restrict=false&sortField=relevance&sortOrder=desc&searchTerm=07df5d38c49a43f69e15e44bd9c4954f#content


Housing elements for San Deigo: 
https://hhubsandiego-ucsdonline.hub.arcgis.com/datasets/205e4290da9e49049529644b2178fbb9_0/explore?location=32.952571%2C-117.066813%2C9.56


San Diego crime rate: 
https://opendata.sandag.org/ARJIS/ARJIS-Public-Crime-Data/3qn6-ugjx/about_data


Shelter location: 
https://www.sandiego.gov/homelessness-strategies-and-solutions/services/shelter
(need to geo encode the address) 


## Data cleaning you have done: 
We imported San Diego (SD) crime data into ArcGIS Online and converted it from a GeoJSON file into a feature layer. We geoencoded the existing homeless shelters and upload it on ArcGIS Online. 

### Import the feature layers we've uploaded to agol.

In [4]:
homeless_mentioned = agol.content.get('07df5d38c49a43f69e15e44bd9c4954f')
sd_crime = agol.content.get('e54740e304f647889d9b86436b70dba5')
# free_food = agol.content.get('a995c4fc08024d30bcf98d12fb326b02')
# crime_rate_hotspots= agol.content.get('0e79d2dde8614860af57df696965c980')
homeless_shelter = agol.content.get('e884228df2a040f7954e956f0367c04f')
# and poverty data used for enrichment that is loaded later on


## Descriptive statistics for the data: 

### Lets visualize all the homeless mentioned in SD.

In [5]:
map1 = agol.map('San Diego')
map1.add_layer(homeless_mentioned)  
map1.add_layer(homeless_mentioned)
map1

MapView(layout=Layout(height='400px', width='100%'))

### Lets also visualize all the crimes reported in SD.

In [6]:
map2 = agol.map('San Diego')
map2.add_layer(sd_crime)  
map2.add_layer(sd_crime)
map2


MapView(layout=Layout(height='400px', width='100%'))

### It is difficult to discern spatial patterns with so many points on the map. To make sense of so many homeless and crime data, we will map them using hot spot analysis. This is help us identify areas of statistically significant clusters.

In [7]:
homeless_hot_spots = find_hot_spots(homeless_mentioned,
                                 output_name='Homeless_Hot_Spots123')

WARNING 001605: Distances for Geographic Coordinates (degrees, minutes, seconds) are analyzed using Chordal Distances in meters.
{"cost": 29.537}


In [8]:
homeless_hot_spots

<Item title:"Homeless_Hot_Spots123" type:Feature Layer Collection owner:dsc170wi24_52>

## Analysis:

### Although there are homeless everywhere all over the map, running hot spot analysis helps us find locations with statistically significant clusters and we can see there is indeed a specific region with a lot more homeless.

In [9]:
homeless_spots_map = agol.map('San Diego')
homeless_spots_map.add_layer(homeless_hot_spots)
homeless_spots_map

MapView(layout=Layout(height='400px', width='100%'))

### Let's also create a hot spot map of crime to compare to the homeless hot spot map

In [10]:
crime_hot_spots = find_hot_spots(sd_crime,
                                         output_name='crime_hot_spot22',
                                         aggregation_polygon_layer=homeless_hot_spots)

WARNING 001605: Distances for Geographic Coordinates (degrees, minutes, seconds) are analyzed using Chordal Distances in meters.
WARNING 001605: Distances for Geographic Coordinates (degrees, minutes, seconds) are analyzed using Chordal Distances in meters.
{"cost": 49.358}


In [11]:
crime_hot_spots

<Item title:"crime_hot_spot22" type:Feature Layer Collection owner:dsc170wi24_52>

In [12]:
crime_hot_spots_lyr = crime_hot_spots.layers[0]

### We also run a hot spot analysis on crime and can see that there is overlap in the hotspot area for homeless and crime.

In [13]:
crime_spots_map =  agol.map('San Diego')
crime_spots_map.add_layer(crime_hot_spots_lyr)
crime_spots_map

MapView(layout=Layout(height='400px', width='100%'))

### We will now get poverty data for each of the polygon grids in the  crime hot spot analysis layer by enriching the data

We will add analysis variables, i.e, poverty, to our layer. This will enrich our layer with the poverty data. 

In [15]:
poverty_enrich= enrich_layer(crime_hot_spots, 
                              analysis_variables=["households.ACSHHBPOV"], 
                              output_name='PovertyDataEnrichedLayer_w2411123')

{"messageCode": "AO_100047", "message": "Enrichment may not be available for some features."}
{"messageCode": "AO_100288", "message": "Unable to detect the country for one or more features."}
{"messageCode": "AO_100047", "message": "Enrichment may not be available for some features."}
{"messageCode": "AO_100000", "message": "Country aggregation mode supports only global data collections. Following data collections couldn't be processed: 'households'"}
{"cost": -1}


In [16]:
poverty_enrich_lyr = poverty_enrich.layers[0]

### Create a hot spot map of poverty

#### We can find hot spots of the enriched_layer by assigning the field name of the analysis variable to parameter 'analysis_field'.

In [17]:
poverty_data_hot_spots = find_hot_spots(poverty_enrich_lyr,
                                        analysis_field='ACSHHBPOV',
                                        output_name='povertyEnrichedHotSpots_w24123')
poverty_data_hot_spots

WARNING 001605: Distances for Geographic Coordinates (degrees, minutes, seconds) are analyzed using Chordal Distances in meters.
{"messageCode": "AO_642", "message": "Problems reading 4 of 1578 total records.", "params": {"badNum": 4, "totalNum": 1578}}
{"messageCode": "AO_848", "message": "Features with bad records (only includes first 30): OBJECTID = 686, 938, 980, 1578.", "params": {"ObjIDs": "OBJECTID", "IDnums": "686, 938, 980, 1578"}}
{"cost": 1.578}


<Item title:"povertyEnrichedHotSpots_w24123" type:Feature Layer Collection owner:dsc170wi24_52>

In [18]:
poverty_data_hot_spots_map = agol.map('San Diego')
poverty_data_hot_spots_map.add_layer(poverty_data_hot_spots)
poverty_data_hot_spots_map

MapView(layout=Layout(height='400px', width='100%'))

### The red areas are statistically significant hot spots for poverty.

### We will now filter the three variables, i.e., homeless, crime, and poverty in order to get statistically significant hot spots. 

We will assign the Gi_Bin attribute a value of 3 in order to get statistically significant hot spots at 99 percent confidence level.

In [19]:
homelesslayer = homeless_hot_spots.layers[0]
homelesslayer.filter = "Gi_Bin = 3"

In [20]:
crimelayer = crime_hot_spots.layers[0]
crimelayer.filter = "Gi_Bin = 3"

In [21]:
povertylayer = poverty_data_hot_spots.layers[0]
povertylayer.filter = "Gi_Bin = 3"

We will use the `overlay_layers` to combine the three layers (homeless, crime, and poverty) to find the hotspots with the most of these to see which areas we should be allocating more resources.

### We will find the intersection of homeless and crime

In [22]:
intersect_of_homeless_crime = overlay_layers(homelesslayer,
                                           crimelayer,
                                           tolerance=0,
                                           context={},
                                           output_name="IntersectOfHomelessCrime_w241123")

{"cost": 0.236}


In [23]:
intersect_of_homeless_crime

<Item title:"IntersectOfHomelessCrime_w241123" type:Feature Layer Collection owner:dsc170wi24_52>

### Now we will overlay all the homeless, crime, and poverty hot spot maps to find where they overlap

In [24]:
intersect_home_cri_pov= overlay_layers(intersect_of_homeless_crime,
                                       poverty_data_hot_spots,
                                       output_name="intersectOfHomelessCrimePoverty_w241123")

{"cost": 0.264}


In [25]:
intersect_home_cri_pov

<Item title:"intersectOfHomelessCrimePoverty_w241123" type:Feature Layer Collection owner:dsc170wi24_52>

## Summary of products and results:


 And load all the intersections onto a map. The blue area identified through our analysis, representing the overlap between homelessness, crime, and poverty, signifies a critical zone where interventions are most needed. This area, defined by its statistically significant hotspots, highlights where the construction of additional shelters and the provision of more treatment opportunities could have the most substantial impact. By targeting this zone, efforts can be more strategically directed to mitigate the factors contributing to crime and improve the quality of life for those experiencing homelessness, aligning with both our project's findings and broader social objectives.

In [26]:
intersected_map = agol.map('San Diego')
intersected_map.add_layer(intersect_home_cri_pov)
intersected_map

MapView(layout=Layout(height='400px', width='100%'))

## Discussion: 
Our project applied GIS technology to explore the connections between homelessness, crime, and poverty, focusing on visualizing these relationships through hotspot mapping. This approach highlighted significant overlaps between areas of high poverty and homelessness, a finding consistent with existing literature suggesting that these factors are closely linked to elevated crime rates.

The intention to use machine learning for predicting suitable locations for homeless shelters shifted due to the insufficient data on existing shelters. Instead, we employed suitability analysis, aligning with recommendations from sources such as CBS8 and the San Diego District Attorney's office, which stress the importance of shelter and treatment for reducing crime among the homeless population.

By mapping out the intersections of homelessness and poverty, our analysis supports the argument for targeted interventions, such as the construction of new shelters, to address urban crime. This methodology not only validates current understandings but also introduces a practical framework for urban planning and social policy aimed at mitigating crime through addressing homelessness and poverty.

## Conclusion

We indeed find a large spot where the homeless, crime, and poverty hot spots overlap.

From background literature and research, it has been shown that homeless and crime in San Diego County go hand in hand, whether those experiencing homelessness is the victim or offender. According to the [CBS8](https://www.cbs8.com/article/news/crime/da-homeless-more-likely-crime-victims-and-perpetrators-sd-county/509-c01ca175-004b-483e-93b1-1b2c66afd5f1), the San Diego District Attory's office mentions that in order to reduce crimes, those experience homelessness "need shelter and treatment in order to live human lives that are free from violence and crime".  

Thus, the blue area we found with overlap between homelessness, crime, and poverty is the area we would want to build more shelters and provide more treatement opportunities. 

Luckily, we also have homeless shelter data and can see how the government's current efforts to tackle homelessness is going. Let's plot the homeless shelter data and see if it aligns with what we have found.

In [27]:
map_all_inter_shelters = agol.map('San Deigo')
map_all_inter_shelters.add_layer(intersect_home_cri_pov)
map_all_inter_shelters.add_layer(homeless_shelter)
map_all_inter_shelters

MapView(layout=Layout(height='400px', width='100%'))

As we can see from the map above, our results do indeed match with the actions the government is working towards as the blue overlapping area of homelessness, crime, and poverty we found have 10 homeless shelters spreaded throughout that area. Another interesting thing to note is that there is also another cluster of shelther more north even though we didn't find any hotspots overlap there. 

This project we carried out can prove to be an effective analysis when figuring out where to build homeless shelters or honestly any resource centers when trying to improve San Diego County.